In [12]:
import sys, site, os
sys.path.insert(0, site.USER_SITE)
os.environ['PYTHONPATH'] = f"{site.USER_SITE}:{os.environ['PYTHONPATH']}"
import uproot  as ur
import math as mt
import numpy as np
import awkward as ak
import vector
import pandas as pd
vector.register_awkward()
import types
from coffea import processor
import coffea
from os import listdir
from os.path import isfile, join
import ROOT


In [22]:
class SimpleAjjEvents :

    def __init__(self , jpt , jeta , drje , mjjcut , photonpt , photoneta , events , lumi ):
        
        self.events = events        
        self.mjj_cut = mjjcut
        self.photonpt_cut = photonpt
        self.photoneta_cut = photoneta
        self.jpt_cut = jpt
        self.jeta_cut = jeta
        self.drje_cut = drje
        self._luminosity = lumi
        #We are spliting this class output into two directory  
        #This directory have events with only two acceptable jets and fills the 3rd jet parameters with non-reasonable value 
        self._accumulator_2jets = processor.dict_accumulator({
                 'j1pt' : np.array(self.jet1_2jet.pt).tolist(),
                 'j1eta' : np.array(self.jet1_2jet.eta).tolist(),
                 'j1m' : np.array(self.jet1_2jet.M).tolist(),
                 'j2pt' : np.array(self.jet2_2jet.pt).tolist(),
                 'j2eta' : np.array(self.jet2_2jet.eta).tolist(),
                 'j2m' : np.array(self.jet2_2jet.M).tolist(),
                 'j1j2pt' : np.array(self.jj_2jet.pt).tolist(),
                 'j1j2deta' : np.array(self.jet1_2jet.eta-self.jet2_2jet.eta).tolist(),
                 'j1j2m' : np.array(self.jj_2jet.M).tolist(),
                 'j1j2dphi' : np.array(self.dphijj_2jet).tolist(),
                 'photonpt': np.array(self.photon_2jet.pt).tolist(),
                 'photoneta':np.array(self.photon_2jet.eta).tolist(),
                 'photonE':np.array(self.photon_2jet.E).tolist(),
                 'photonm':np.array(abs(self.photon_2jet.M)).tolist(),
                 'D' : np.array(self.D_2jet).tolist(),
                 'aplanarity' : np.array(self.aplanarity_2jet).tolist(),
                 'C' : np.array(self.C_2jet).tolist(),
                 'circularity' : np.array(self.circularity_2jet).tolist(),
                 'zep' : np.array(self.photon_2jet.eta - ((self.jet1_2jet.eta + self.jet2_2jet.eta) / 2)).tolist(),
                 'zep_dijet': np.array((self.photon_2jet.eta - ((self.jet1_2jet.eta + self.jet2_2jet.eta) / 2))/(self.jet1_2jet.eta-self.jet2_2jet.eta)).tolist(),
                 'j3pt':np.full_like(np.array(self.jet1_2jet.pt),-999).tolist(),
                 'j3eta': np.full_like(np.array(self.jet1_2jet.eta),-999).tolist(),
                 'j3m': np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j1j3dphi' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j2j3dphi' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j1j3deta' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j2j3deta' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j1j3m' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j2j3m' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'j3ptratio' : np.full_like(np.array(self.jet1_2jet.M),-999).tolist(),
                 'weights' : np.array(self.weights).tolist()
                
        })
        #This directory have events with more than three acceptable jets but only keeps prameters of three jets 
        self._accumulator_3jets = processor.dict_accumulator({
                 'j1pt' : np.array(self.jet1_3jet.pt).tolist(),
                 'j1eta' : np.array(self.jet1_3jet.eta).tolist(),
                 'j1m' : np.array(self.jet1_3jet.M).tolist(),
                 'j2pt' : np.array(self.jet2_3jet.pt).tolist(),
                 'j2eta' : np.array(self.jet2_3jet.eta).tolist(),
                 'j2m' : np.array(self.jet2_3jet.M).tolist(),
                 'j1j2pt' : np.array(self.j1j2_3jet.pt).tolist(),
                 'j1j2deta' : np.array(abs(self.jet1_3jet.eta-self.jet2_3jet.eta)).tolist(),
                 'j1j2m' : np.array(self.j1j2_3jet.M).tolist(),
                 'j1j2dphi' : np.array(self.dphij1j2_3jet).tolist(),
                 'photonpt': np.array(self.photon_3jet.pt).tolist(),
                 'photoneta':np.array(self.photon_3jet.eta).tolist(),
                 'photonE':np.array(self.photon_3jet.E).tolist(),
                 'photonm':np.array(abs(self.photon_3jet.M)).tolist(),
                 'D' : np.array(self.D_3jet).tolist(),
                 'aplanarity' : np.array(self.aplanarity_3jet).tolist(),
                 'C' : np.array(self.C_3jet).tolist(),
                 'circularity' : np.array(self.circularity_3jet).tolist(),
                 'zep' : np.array(self.photon_3jet.eta - ((self.jet1_3jet.eta + self.jet2_3jet.eta) / 2)).tolist(),
                 'zep_dijet': np.array((self.photon_3jet.eta - ((self.jet1_3jet.eta + self.jet2_3jet.eta) / 2))/(self.jet1_3jet.eta-self.jet2_3jet.eta)).tolist(),
                 'j3pt' : np.array(self.jet3_3jet.pt).tolist(),
                 'j3eta' : np.array(self.jet3_3jet.eta).tolist(),
                 'j3m' : np.array(self.jet3_3jet.M).tolist(),
                 'j1j3dphi' : np.array(self.dphij1j3_3jet).tolist(),
                 'j2j3dphi' : np.array(self.dphij2j3_3jet).tolist(),
                 'j1j3deta' : np.array(self.jet1_3jet.eta-self.jet3_3jet.eta).tolist(),
                 'j2j3deta' : np.array(self.jet2_3jet.eta-self.jet3_3jet.eta).tolist(),
                 'j1j3m' : np.array(self.j1j3_3jet.M).tolist(),
                 'j2j3m' : np.array(self.j2j3_3jet.M).tolist(),
                 'j3ptratio' : np.array((self.jet3_3jet.pt)/(self.photon_3jet.pt + self.jet1_3jet.pt + self.jet2_3jet.pt )).tolist(),
                 'weights' : np.array(self.weights_3jets).tolist()
                })
    @property
    #We put our two directory in one 
    def accumulator(self):
        self.dic_acm=processor.dict_accumulator({
             '2jets' : self._accumulator_2jets, 
             '3jets' : self._accumulator_3jets
         })
        return self.dic_acm

    @property
    def events(self):
        return self._events
    @events.setter
    def events(self,v):
        self._events = v
        for attr in ('_cleanJets','_jetsp4','_elecsp4','_selectedJets','_selectedEvents', '_thephoton' , '_jet1' , '_jet2' , '_jj','_dphijj','_photons','__weights','__selectionStep1'):
            self.__dict__.pop(attr,None)

    ####DATASET INFO
    @property
    def dataset(self):
        return self.events.metadata['dataset']
    
    @property
    def xsection(self):
        ds = self.dataset
        #'725ToInf',  '100To200','50To100','400To725','200To400' 
        if ds == 'signal':
            return 12
        elif '50To100' in ds:
            return 846
        elif '100To200' in ds:
            return 141.1
        elif '200To400' in ds:
            return 25.79
        elif '400To725' in ds:
            return 1.082
        elif '725To' in ds:
            return 0.53
    @property
    def kFactor(self):
        ds = self.dataset
        
        #'725ToInf',  '100To200','50To100','400To725','200To400' 
        if ds == 'signal':
            return 1
        elif '50To100' in ds:
            return 7
        elif '100To200' in ds:
            return 7
        elif '200To400' in ds:
            return 1
        elif '400To725' in ds:
            return 1
        elif '725To' in ds:
            return 0.4
    @property
    def luminosity(self):
        return self._luminosity
    @property
    def filterEff(self):
        ds = self.dataset
        #'725ToInf',  '100To200','50To100','400To725','200To400' 
        if ds == 'signal':
            return 1
        elif '50To100' in ds:
            return 0.03
        elif '100To200' in ds:
            return 0.02
        elif '200To400' in ds:
            return 0.6
        elif '400To725' in ds:
            return 0.73
        elif '725To' in ds:
            return 0.67
    @property
    def weight(self):
        return self.luminosity*self.xsection*self.filterEff*self.kFactor

    @property
    def nWeights(self):
        if not hasattr(self , '_nWeights'):
            self._nWeights = len( self.events.Weight[0,:] )
        return self._nWeights
    
    @property
    def weights(self):
        if not hasattr(self , '_weights'):
            self._weights = self.events.genWeight[self.selectionStep1,:][self.selectedEvents,:]
        return self._weights
    @property
    def weights_3jets(self):
        if not hasattr(self , '_weights_3jets'):
            self._weights_3jets = self.events.genWeight[self.selectionStep1_3jets,:][self.selectedEvents_3jets,:]
        return self._weights_3jets

    ###############################################
    ####SELECTION
    @property
    def photons(self):
        if not hasattr(self , '_photons'):
            allphotons = ak.zip({'PT': self.events.Photon_pt, 'Eta': self.events.Photon_eta, 'Phi': self.events.Photon_phi, 'E': self.events.Photon_eCorr , 'M' : self.events.Photon_mass },with_name= "Momentum4D")
            selected_photons = (allphotons.PT > self.photonpt_cut) & (abs(allphotons.Eta) < self.photoneta_cut )
            self._photons = allphotons[selected_photons]
        return self._photons
    @property
    def has1photon(self):
        return ak.num(self.photons)>0
    @property
    def jets(self):
        self._jets = ak.zip({'PT': self.events.Jet_pt, 'Eta': self.events.Jet_eta, 'Phi': self.events.Jet_phi, 'M': self.events.Jet_mass },with_name= "Momentum4D")
        return self._jets
    @property
    def jetsp4(self):
        if not hasattr(self, '_jetsp4'):
            self._jetsp4 = ak.zip({'pt': self.events.Jet_pt, 'eta': self.events.Jet_eta, 'phi': self.events.Jet_phi, 'mass': self.events.Jet_mass },with_name= "Momentum4D")
        return self._jetsp4
    @property
    def elecsp4(self):
        if not hasattr(self, '_elecsp4'):
            self._elecsp4 = ak.zip({'pt': self.events.Electron_pt, 'eta': self.events.Electron_eta, 'phi': self.events.Electron_phi, 'charge': self.events.Electron_charge},with_name= "Momentum4D")
        return self._elecsp4
    @property
    def cleanJets(self):
        if not hasattr(self,'_cleanJets'):
            elecs = self.elecsp4
            jet_electron = ak.unzip( ak.cartesian([self.jetsp4,elecs],axis=1,nested=True, with_name= "Momentum4D") )
            jele_dr = jet_electron[0].deltaR(jet_electron[1])
            jele_mindr = ak.min( jele_dr , axis=2 ,mask_identity=False, initial=100)
            jpt_cut = self.jets.PT > self.jpt_cut
            jeta_cut= abs(self.jets.Eta) < self.jeta_cut
            jmindre_cut = jele_mindr > self.drje_cut
            
            self._selectedJets = jpt_cut & jeta_cut & jmindre_cut
            self._cleanJets = self.jetsp4[self._selectedJets]
            
        return self._cleanJets
    
    @property
    def selectionStep1(self):
        if not hasattr(self , "_selectionStep1"):
            self._selectionStep1 = (ak.num(self.cleanJets)==2) & self.has1photon
        return self._selectionStep1
        
    @property
    def selectedEvents(self):
        if not hasattr(self , "_selectedEvents"):
            _jet1 = self.cleanJets[self.selectionStep1][:,0]
            _jet2 = self.cleanJets[self.selectionStep1][:,1]
            _jj = _jet1+_jet2
            self._selectedEvents = (_jj.M > self.mjj_cut) & (abs(_jet1.eta - _jet2.eta) > 3.0 )
        return self._selectedEvents
    @property
    def nSelectedEvents(self):
        return ak.sum(self.selectedEvents)

    @property
    def selectionStep1_3jets(self):
        if not hasattr(self , "_selectionStep1_3jets"):
            self._selectionStep1_3jets = (ak.num(self.cleanJets)>=3) & self.has1photon
        return self._selectionStep1_3jets
    @property
    def selectedEvents_3jets(self):
        if not hasattr(self , "_selectedEvents_3jets"):
            _jet1 = self.cleanJets[self.selectionStep1_3jets][:,0]
            _jet2 = self.cleanJets[self.selectionStep1_3jets][:,1]
            _jj = _jet1+_jet2
            self._selectedEvents_3jets = (_jj.M > self.mjj_cut) & (abs(_jet1.eta - _jet2.eta) > 3.0 )
        return self._selectedEvents_3jets
    ###############################################
    ####MAIN INGEREDIENTS for events with 2 jet
    @property
    def photon_2jet(self):
        if not hasattr(self , "_thephoton_2jet"):
            _photons = ak.zip({'pt': self.events.Photon_pt, 'eta': self.events.Photon_eta, 'phi': self.events.Photon_phi, 'e': self.events.Photon_eCorr , 'm' : self.events.Photon_mass },with_name= "Momentum4D")
            self._thephoton_2jet = _photons[self.selectionStep1][self.selectedEvents,0]
        return self._thephoton_2jet
    @property
    def jet1_2jet(self):
        if not hasattr(self , '_jet1_2jet'):
            self._jet1_2jet = self.cleanJets[self.selectionStep1][self.selectedEvents,0]
        return self._jet1_2jet
    @property
    def jet2_2jet(self):
        if not hasattr(self , '_jet2_2jet'):
            self._jet2_2jet = self.cleanJets[self.selectionStep1][self.selectedEvents,1]
        return self._jet2_2jet  
        
    @property
    def jj_2jet(self):
        if not hasattr(self , '_jj_2jet'):
            
            self._jj_2jet = self.jet1_2jet + self.jet2_2jet
            
        return self._jj_2jet
    #################################################
    ####MAIN INGEREDIENTS for events with 3 jet
    @property
    def photon_3jet(self):
        if not hasattr(self , "_thephoton_3jet"):
            _photons = ak.zip({'pt': self.events.Photon_pt, 'eta': self.events.Photon_eta, 'phi': self.events.Photon_phi, 'e': self.events.Photon_eCorr, 'm' : self.events.Photon_mass},with_name= "Momentum4D")
            self._thephoton_3jet = _photons[self.selectionStep1_3jets][self.selectedEvents_3jets,0]
        return self._thephoton_3jet
    @property
    def jet1_3jet(self):
        if not hasattr(self , '_jet1_3jet'):
            self._jet1_3jet = self.cleanJets[self.selectionStep1_3jets][self.selectedEvents_3jets,0]
        return self._jet1_3jet
    @property
    def jet2_3jet(self):
        if not hasattr(self , '_jet2_3jet'):
            self._jet2_3jet = self.cleanJets[self.selectionStep1_3jets][self.selectedEvents_3jets,1]
        return self._jet2_3jet
    @property
    def jet3_3jet(self):
        if not hasattr(self , '_jet3_3jet'):
            self._jet3_3jet  = self.cleanJets[self.selectionStep1_3jets][self.selectedEvents_3jets,2]
        return self._jet3_3jet    
        
    @property
    def j1j2_3jet(self):
        if not hasattr(self , '_j1j2_3jet'):
            self._j1j2_3jet = self.jet1_3jet+self.jet2_3jet
            
        return self._j1j2_3jet
    @property
    def j1j3_3jet(self):
        if not hasattr(self , '_j1j3_3jet'):
            self._j1j3_3jet = self.jet1_3jet+self.jet3_3jet
            
        return self._j1j3_3jet
    @property
    def j2j3_3jet(self):
        if not hasattr(self , '_j2j3_3jet'):
            self._j2j3_3jet = self.jet2_3jet+self.jet3_3jet
            
        return self._j2j3_3jet
    
    #################################################
    ####SECONDARY INPUTS
    @property
    def eventShapeEigs_2jet(self):
        if not hasattr(self,'_eventShapeEigs_2jet'):
            t00 = self.jet1_2jet.px*self.jet1_2jet.px + self.jet2_2jet.px*self.jet2_2jet.px + self.photon_2jet.px*self.photon_2jet.px
            t01 = self.jet1_2jet.px*self.jet1_2jet.py + self.jet2_2jet.px*self.jet2_2jet.py + self.photon_2jet.px*self.photon_2jet.py
            t02 = self.jet1_2jet.px*self.jet1_2jet.pz + self.jet2_2jet.px*self.jet2_2jet.pz + self.photon_2jet.px*self.photon_2jet.pz
            
            t11 = self.jet1_2jet.py*self.jet1_2jet.py + self.jet2_2jet.py*self.jet2_2jet.py + self.photon_2jet.py*self.photon_2jet.py
            t12 = self.jet1_2jet.py*self.jet1_2jet.pz + self.jet2_2jet.py*self.jet2_2jet.pz + self.photon_2jet.py*self.photon_2jet.pz
            
            t22 = self.jet1_2jet.pz*self.jet1_2jet.pz + self.jet2_2jet.pz*self.jet2_2jet.pz + self.photon_2jet.pz*self.photon_2jet.pz
            
            norm = t00 + t11 + t22
            tensor = np.array([[t00/norm,t01/norm,t02/norm],
                              [t01/norm,t11/norm,t12/norm],
                              [t02/norm,t12/norm,t22/norm]])
            #print('a', tensor.shape)
            tensor_sw = np.swapaxes(tensor,0,2)
            #print('b', tensor_sw.shape)
            self._eventShapeEigs_2jet = np.linalg.eigvalsh(tensor_sw)

            
        return self._eventShapeEigs_2jet
    
    @property
    def eventShapeEigs_3jet(self):
        if not hasattr(self,'_eventShapeEigs_3jet'):
            t00 = self.jet1_3jet.px*self.jet1_3jet.px + self.jet2_3jet.px*self.jet2_3jet.px + self.photon_3jet.px*self.photon_3jet.px
            t01 = self.jet1_3jet.px*self.jet1_3jet.py + self.jet2_3jet.px*self.jet2_3jet.py + self.photon_3jet.px*self.photon_3jet.py
            t02 = self.jet1_3jet.px*self.jet1_3jet.pz + self.jet2_3jet.px*self.jet2_3jet.pz + self.photon_3jet.px*self.photon_3jet.pz
            
            t11 = self.jet1_3jet.py*self.jet1_3jet.py + self.jet2_3jet.py*self.jet2_3jet.py + self.photon_3jet.py*self.photon_3jet.py
            t12 = self.jet1_3jet.py*self.jet1_3jet.pz + self.jet2_3jet.py*self.jet2_3jet.pz + self.photon_3jet.py*self.photon_3jet.pz
            
            t22 = self.jet1_3jet.pz*self.jet1_3jet.pz + self.jet2_3jet.pz*self.jet2_3jet.pz + self.photon_3jet.pz*self.photon_3jet.pz
            
            norm = t00 + t11 + t22
            tensor = np.array([[t00/norm,t01/norm,t02/norm],
                              [t01/norm,t11/norm,t12/norm],
                              [t02/norm,t12/norm,t22/norm]])
            #print('a', tensor.shape)
            tensor_sw = np.swapaxes(tensor,0,2)
            #print('b', tensor_sw.shape)
            self._eventShapeEigs_3jet = np.linalg.eigvalsh(tensor_sw)

            
        return self._eventShapeEigs_3jet
    
    @property
    def circularity_2jet(self):
        if not hasattr(self , "_circularity"):
            area = self.jet1_2jet.pt + self.jet2_2jet.pt + self.photon_2jet.pt
            cosphi = np.cos( np.linspace(0 , 2*np.pi , 100) )
            sinphi = np.sin( np.linspace(0 , 2*np.pi , 100) )
            vals = np.abs( np.outer( self.photon_2jet.px ,cosphi ) + np.outer( self.photon_2jet.py ,sinphi ) ) + np.abs( np.outer( self.jet1_2jet.px ,cosphi ) + np.outer( self.jet1_2jet.py ,sinphi ) ) + np.abs( np.outer( self.jet2_2jet.px ,cosphi ) + np.outer( self.jet2_2jet.py ,sinphi ) ) 
            self._circularity_2jet = np.pi * np.min(vals,1) / (2*area)
        return self._circularity_2jet
    @property
    def circularity_3jet(self):
        if not hasattr(self , "_circularity_3jet"):
            area = self.jet1_3jet.pt + self.jet2_3jet.pt + self.photon_3jet.pt
            cosphi = np.cos( np.linspace(0 , 2*np.pi , 100) )
            sinphi = np.sin( np.linspace(0 , 2*np.pi , 100) )
            vals = np.abs( np.outer( self.photon_3jet.px ,cosphi ) + np.outer( self.photon_3jet.py ,sinphi ) ) + np.abs( np.outer( self.jet1_3jet.px ,cosphi ) + np.outer( self.jet1_3jet.py ,sinphi ) ) + np.abs( np.outer( self.jet2_3jet.px ,cosphi ) + np.outer( self.jet2_3jet.py ,sinphi ) ) 
            self._circularity_3jet = np.pi * np.min(vals,1) / (2*area)
        return self._circularity_3jet
    @property
    def dphijj_2jet(self):
        if not hasattr(self , '_dphijj_2jet'):
            self._dphijj_2jet = abs( self.jet1_2jet.deltaphi(self.jet2_2jet) )
        return self._dphijj_2jet
    @property
    def dphij1j2_3jet(self):
        if not hasattr(self , '_dphij1j2_3jet'):
            self._dphij1j2_3jet = abs( self.jet1_3jet.deltaphi(self.jet2_3jet) )
        return self._dphij1j2_3jet
    @property
    def dphij1j3_3jet(self):
        if not hasattr(self , '_dphij1j3_3jet'):
            self._dphij1j3_3jet = abs( self.jet1_3jet.deltaphi(self.jet3_3jet) )
        return self._dphij1j3_3jet
    @property
    def dphij2j3_3jet(self):
        if not hasattr(self , '_dphij2j3_3jet'):
            self._dphij2j3_3jet = abs( self.jet2_3jet.deltaphi(self.jet3_3jet) )
        return self._dphij2j3_3jet
    @property
    def aplanarity_2jet(self):
        if not hasattr(self , '_aplanarity_2jet'):
            self._aplanarity_2jet =  1.5*self.eventShapeEigs_2jet[:,0]
        return self._aplanarity_2jet
    
    @property
    def aplanarity_3jet(self):
        if not hasattr(self , '_aplanarity_3jet'):
            self._aplanarity_3jet =  1.5*self.eventShapeEigs_3jet[:,0]
        return self._aplanarity_3jet
        
    @property
    def dphivj1_2jet(self):
        if not hasattr(self , '_dphivj1_2jet'):
            self._dphivj1_2jet = abs( self.jet2_2jet.deltaphi(self.photon_2jet) )
        return self._dphivj1_2jet
    @property
    def dphivj1_3jet(self):
        if not hasattr(self , '_dphivj1_3jet'):
            self._dphivj1_3jet = abs( self.jet2_3jet.deltaphi(self.photon_3jet) )
        return self._dphivj1
    @property
    def detajj_2jet(self):
        if not hasattr(self , '_detajj_2jet'):
            self._detajj_2jet = abs(self.jet1_2jet.deltaeta(self.jet2_2jet))
        return self._detajj_2jet
    @property
    def detajj_3jet(self):
        if not hasattr(self , '_detajj_3jet'):
            self._detajj_3jet = abs(self.jet1_3jet.deltaeta(self.jet2_3jet))
        return self._detajj_3jet

    @property
    def drjj_2jet(self):
        if not hasattr(self , '_drjj_2jet'):
            self._drjj_2jet = self.jet1_2jet.deltaR(self.jet2_2jet)
        return self._drjj_2jet
    @property
    def drjj_3jet(self):
        if not hasattr(self , '_drjj_3jet'):
            self._drjj_3jet = self.jet1_3jet.deltaR(self.jet2_3jet)
        return self._drjj_3jet
    @property
    def C_2jet(self):
        if not hasattr(self , '_C_2jet'):
            self._C_2jet =  3. * (self.eventShapeEigs_2jet[:,0] * self.eventShapeEigs_2jet[:,1] + self.eventShapeEigs_2jet[:,0] * self.eventShapeEigs_2jet[:,2] + self.eventShapeEigs_2jet[:,1] * self.eventShapeEigs_2jet[:,2])
        return self._C_2jet

    @property
    def C_3jet(self):
        if not hasattr(self , '_C_3jet'):
            self._C_3jet =  3. * (self.eventShapeEigs_3jet[:,0] * self.eventShapeEigs_3jet[:,1] + self.eventShapeEigs_3jet[:,0] * self.eventShapeEigs_3jet[:,2] + self.eventShapeEigs_3jet[:,1] * self.eventShapeEigs_3jet[:,2])
        return self._C_3jet
    @property
    def D_2jet(self):
        if not hasattr(self , '_D_2jet'):
            self._D_2jet =   27. * self.eventShapeEigs_2jet[:,0] * self.eventShapeEigs_2jet[:,1] * self.eventShapeEigs_2jet[:,2]
        return self._D_2jet

    @property
    def D_3jet(self):
        if not hasattr(self , '_D_3jet'):
            self._D_3jet =   27. * self.eventShapeEigs_3jet[:,0] * self.eventShapeEigs_3jet[:,1] * self.eventShapeEigs_3jet[:,2]
        return self._D_3jet
    @property
    def dphivj0_2jet(self):
        if not hasattr(self , '_dphivj0_2jet'):
            self._dphivj0_2jet = abs( self.jet1_2jet.deltaphi(self.photon_2jet) )
        return self._dphivj0_2jet
    @property
    def dphivj0_3jet(self):
        if not hasattr(self , '_dphivj0_3jet'):
            self._dphivj0_3jet = abs( self.jet1_3jet.deltaphi(self.photon_3jet) )
        return self._dphivj0_3jet
        
    @property
    def j_qg1_2jet(self):
        if not hasattr(self , '_j_qg1_2jet'):
            jf = abs(self.jet2_2jet.flavor) < 7
            self._j_qg1_2jet =  jf*0.9 + np.logical_not(jf)*0.1
        return self._j_qg1_2jet

    @property
    def j_qg0_2jet(self):
        if not hasattr(self , '_j_qg0_2jet'):
            jf = abs(self.jet1_2jet.flavor) < 7
            self._j_qg0_2jet =  jf*0.9 + np.logical_not(jf)*0.1
        return self._j_qg0_2jet

    @property
    def ystar_2jet(self):
        if not hasattr(self , '_ystar_2jet'):
            self._ystar_2jet = self.photon_2jet.eta-0.5*self.jj_2jet.eta
        return self._ystar_2jet
    @property
    def relbpt_2jet(self):
        if not hasattr(self , '_relbpt_2jet'):
            self._relbpt_2jet = (self.jet1_2jet.pt + self.jet2_2jet.pt) / self.photon_2jet.pt
        return self._relbpt_2jet
    
    @property
    def dphibjj_2jet(self):
        if not hasattr(self , '_dphibjj_2jet'):
            self._dphibjj_2jet = abs( self.jj_2jet.deltaphi(self.photon_2jet) )
        return self._dphibjj_2jet
    
    
    @property
    def j_qg1_3jet(self):
        if not hasattr(self , '_j_qg1_3jet'):
            jf = abs(self.jet2_3jet.flavor) < 7
            self._j_qg1_3jet =  jf*0.9 + np.logical_not(jf)*0.1
        return self._j_qg1_3jet

    @property
    def j_qg0_3jet(self):
        if not hasattr(self , '_j_qg0_3jet'):
            jf = abs(self.jet1_3jet.flavor) < 7
            self._j_qg0_3jet =  jf*0.9 + np.logical_not(jf)*0.1
        return self._j_qg0_3jet

    @property
    def ystar_3jet(self):
        if not hasattr(self , '_ystar_3jet'):
            self._ystar_3jet = self.photon_3jet.eta-0.5*self.j1j2_3jet.eta
        return self._ystar_3jet
    @property
    def relbpt_3jet(self):
        if not hasattr(self , '_relbpt_3jet'):
            self._relbpt_3jet = (self.jet1_3jet.pt + self.jet2_3jet.pt) / self.photon_3jet.pt
        return self._relbpt_3jet
    
    @property
    def dphibjj_3jet(self):
        if not hasattr(self , '_dphibjj_3jet'):
            self._dphibjj_3jet = abs( self.j1j2_3jet.deltaphi(self.photon_3jet) )
        return self._dphibjj_3jet

    
    
class AjjProcessor(processor.ProcessorABC):
    def __init__(self , mjjcut , photonpt , photoneta , jpt , jeta , drje , lumi ,  foutname, sample):
        self.mjjcut = mjjcut
        self.photonpt = photonpt 
        self.photoneta = photoneta
        
        self.jpt = jpt
        self.jeta = jeta
        self.drje =  drje
        self.lumi = lumi
        
        self.foutname = foutname
        self.sample = sample

    def process(self, events):
        ajjevents = SimpleAjjEvents(self.jpt , self.jeta , self.drje , self.mjjcut , self.photonpt , self.photoneta , events , self.lumi)
        return ajjevents.accumulator
        
    def postprocess(self, accumulator):
        lay1=0
        for h in accumulator:
            lay2=0
            
            for l in accumulator[h]:
                if lay2==0:
                    array = np.asarray(accumulator[h][l])
                    lay2+=1
                else:
                    array =  np.c_[array,np.asarray(accumulator[h][l])]
            if  lay1==0:
                self.results = array
                lay1+=1
            else:
                self.results =  np.r_[self.results,array]
        np.save(self.foutname,self.results)
        return accumulator


In [23]:
a=AjjProcessor(mjjcut=50 , photonpt=70 , photoneta=1.47 , jpt=50 , jeta=4.5 , drje=0.4, lumi=3000, sample='signal', foutname='sig1.npy')

def onlyfiles(mypath , nfirst=40):
    if nfirst == 0:
        return [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    else:
        return [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

fileset ={
    'signal' : onlyfiles('/eos/user/m/mkaramsi/SWAN_projects/VBF_signal')

}

iterative_run = processor.Runner(
    executor = processor.FuturesExecutor(workers=20),
    schema = coffea.nanoevents.BaseSchema,
     maxchunks=10
    )
iter_out = iterative_run(
    fileset,
    treename="Events",
    processor_instance=a
)

Output()

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/eos/user/m/mkaramsi/.local/lib/python3.8/site-packages/coffea/processor/executor.py", line 1631, in _work_function
    out = processor_instance.process(events)
  File "<ipython-input-22-74fba4fb9337>", line 565, in process
    ajjevents = SimpleAjjEvents(self.jpt , self.jeta , self.drje , self.mjjcut , self.photonpt , self.photoneta , events , self.lumi)
  File "<ipython-input-22-74fba4fb9337>", line 46, in __init__
    'weights' : np.array(self.weights).tolist()
  File "<ipython-input-22-74fba4fb9337>", line 171, in weights
    self._weights = self.events.genWeight[self.selectionStep1,:][self.selectedEvents,:]
  File "/eos/user/m/mkaramsi/.local/lib/python3.8/site-packages/awkward/highlevel.py", line 991, in __getitem__
    tmp = ak._util.wrap(self.layout[where], self._behavior)
ValueError: in NumpyArray, too many dimensions in slice

(https://github.com/scikit-hep/awkward-1.0/blob/1.10.1/src

Exception: Failed processing file: WorkItem(dataset='signal', filename='/eos/user/m/mkaramsi/SWAN_projects/VBF_signal/sig51.root', treename='Events', entrystart=768000, entrystop=864000, fileuuid=b'|\xcf\xe3l^z\x11\xed\x86\x0b\x04\xbf\xe1\x83\xbe\xef', usermeta={})

In [15]:
a=AjjProcessor(mjjcut=50 , photonpt=70 , photoneta=1.47 , jpt=50 , jeta=4.5 , drje=0.4, lumi=3000, sample='background', foutname='back1.npy')

def onlyfiles(mypath , nfirst=40):
    if nfirst == 0:
        return [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    else:
        return [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]

fileset ={
     'background':onlyfiles('/eos/user/m/mkaramsi/SWAN_projects/VBF_back')
}

iterative_run = processor.Runner(
    executor = processor.FuturesExecutor(workers=20),
    schema = coffea.nanoevents.BaseSchema,
     maxchunks=10
    )
iter_out = iterative_run(
    fileset,
    treename="Events",
    processor_instance=a
)

Output()